# Quantum Circuit Simulator

In [17]:
import numpy as np
from numpy.random import choice

### Define unitary matrices

In [19]:
isqrt2 = 1/np.sqrt(2) # inverse of the square root of 2

x = np.array([
    [0, 1],
    [1, 0]
])

z = np.array([
    [1, 0],
    [0, -1]
])

h = np.array([
    [isqrt2, isqrt2],
    [isqrt2, -isqrt2],
])

cx = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0],
])

sw = np.array([
    [1, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 1],
])

unitary_gates = {"x": x, "h": h, "cx": cx, "sw": sw}

### Computational basis vector states

In [20]:
# computational basis
basis_states = [ [1, 0], [0, 1] ]

# classical bit states as strings
cbits = ["0", "1"]

### Quantum Circuit Simulator class with methods required for simulation

In [24]:
class QCSimulator:
    def __init__(self, qubits_num, shots, qcirc):
        """
        Takes a quantum circuit program, qcirc. Creates initial state vector
        of qubits_num qubits.
        """
        # initializer
        self.qubits_num = qubits_num
        self.shots_num = shots
        
        # create initial state vector of qubits_num qubits [1, 0, ... ]
        self.init_state = self.get_ground_state()
        print("Initial state = ", self.init_state)
        
        # number of gate operations to compute
        operations_num = len(qcirc)
        
        # circuit handler, read circuit, calculate matrix operator
        # multiply the state with the operator and return final state.
        final_state = self.circuit_handler(operations_num, qcirc)
        print("final_state = ", final_state)
        
        # make measurements, get counts
        counts = self.get_counts(final_state)
        print(counts)
        
    
    def get_ground_state(self):
        """Initialize a quantum state.
        Returns a vector of size 2**qubits_num with all zeroes except
        the first element which is 1."""
        ground_state_base = np.array([1, 0])
        ground_state_qnum = ground_state_base
        for i in range(self.qubits_num - 1):
            ground_state_qnum = np.kron(ground_state_base, ground_state_qnum)
            
        return ground_state_qnum
    
    
    def get_operator(self, gate_unitary, qubits_target):
        """Returns a unitary operator of size 2**n x 2**n for a given
        quantum gate and target number of qubits.
        Resizing the gate's martix to the dimension of the state vector"""
        
        # define 2x2 identity matrix
        I = np.identity(2)
        
        gate_unitary_post = gate_unitary
        for i in range(self.qubits_num - 1):
            # check if operator size/shape is equal to 2**n x 2**n, where n = qubits_num
            if np.shape(gate_unitary_post) == (2**self.qubits_num, 2**self.qubits_num):
                return gate_unitary_post
            
            if qubits_target == 0:
                gate_unitary_post = np.kron(gate_unitary_post, I)
            elif qubits_target == 1:
                gate_unitary_post = np.kron(I, gate_unitary_post)
        
        return gate_unitary_post
    
    
    def circuit_handler(self, operations_num, qcirc):
        """Handles quantum circuit input, managing operations with gates, operators
        and state vectors."""
        final_state = self.init_state
        
        for i in range(operations_num):
            gate_name = qcirc[i]["gate"]
            target = qcirc[i]["target"]
            if len(target) == 1:
                target = target[0]
            elif len(target) == 2:
                target = target[1]
            print("target: ", target)
            
            if gate_name in unitary_gates.keys():
                gate = unitary_gates[gate_name]
            
            # resize gates
            gate = self.get_operator(gate, target)
            print("%s-gate\n\b%s"%(gate_name, gate))
            
            # multiply states with operators
            try:
                final_state = np.dot(gate, final_state)
            except ValueError:
                print("\n[-] Error: Unitary gate shape: {} not aligned with state vector shape: {}\n".format(np.shape(gate), np.shape(final_state)))
                break
                
            print("final state = ", final_state)
            
        return final_state
                    
        
    
    #def measure_all(self, state_vector, basis_vector):
        """Chooses an element from the state vector using weighted random technique and
        returns it's index."""
        
        
            
    
    
    def get_counts(self, state_vector):
        """Execute measurements in a loop shots_num times and returns object
        with statistics in the form:
        {
            element_index: number_of_occurrences,
            ...
        }
        only from elements which occurred - returned from measrements."""
        # find list of all possible quantum states
        qstates = basis_states
        cstates = cbits
        for i in range(self.qubits_num - 1):
            qstates = [ list(np.kron(j, k)) for j in basis_states for k in qstates ]
            cstates = [ j + k for j in cbits for k in cstates ]
            
        print(qstates)
        print(cstates)
        
        # fill in probabilities of each quantum state
        prob_states = []
        for i in range( len(qstates) ):
            prob_states.append( np.dot( qstates[i], state_vector )**2 )
            
        print(prob_states)
        
        # loop measurement using weighted random technique
        # dictionary to hold statistics of states and their counts
        counts = {}
        counter = 0
        for i in range( self.shots_num ):
            collapsed_state = choice( cstates, 1, p=prob_states )
            if collapsed_state[0] in counts:
                counts[collapsed_state[0]] += 1
            elif not collapsed_state[0] in counts:
                counts[collapsed_state[0]] = 1
                
        # print(counts)
        return counts

    

N = 2   # number of qubits for the state vector; 2-qubits state vector
shots = 1024    # number of executions

# quantum circuit/program
circ = [
    {"gate": "x", "target": [1]},
    #{"gate": "x", "target": [0]},
    {"gate": "h", "target": [0]},
    {"gate": "cx", "target": [0, 1]},
]

QCSimulator(N, shots, circ)

Initial state =  [1 0 0 0]
target:  1
x-gate
[[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]
final state =  [0. 1. 0. 0.]
target:  0
h-gate
[[ 0.70710678  0.          0.70710678  0.        ]
 [ 0.          0.70710678  0.          0.70710678]
 [ 0.70710678  0.         -0.70710678 -0.        ]
 [ 0.          0.70710678 -0.         -0.70710678]]
final state =  [0.         0.70710678 0.         0.70710678]
target:  1
cx-gate
[[1 0 0 0]
 [0 1 0 0]
 [0 0 0 1]
 [0 0 1 0]]
final state =  [0.         0.70710678 0.70710678 0.        ]
final_state =  [0.         0.70710678 0.70710678 0.        ]
[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
['00', '01', '10', '11']
[0.0, 0.4999999999999999, 0.4999999999999999, 0.0]
{'01': 490, '10': 534}
